In [30]:
from langchain_upstage import ChatUpstage
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from tool import news_tool
from dotenv import load_dotenv
from pprint import pprint
import os

from langchain.globals import set_debug

set_debug(True)

load_dotenv()
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")

chat = ChatUpstage(model="solar-pro2")
response = chat.invoke("Hello, how are you?")

[llm/start] [llm:ChatUpstage] Entering LLM run with input:
{
  "prompts": [
    "Human: Hello, how are you?"
  ]
}
[llm/end] [llm:ChatUpstage] [1.15s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Hello! I'm just a large language model, so I don't have feelings, but I'm here and ready to help you. How can I assist you today? Is there something you'd like to talk about or ask? 😊",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Hello! I'm just a large language model, so I don't have feelings, but I'm here and ready to help you. How can I assist you today? Is there something you'd like to talk about or ask? 😊",
            "ad

### Chain으로 최신 뉴스 가져오고 -> 유저 요청에 맞게 응답!

In [31]:
from prompts import SYSTEM_INSTRUCTION

prompt = ChatPromptTemplate.from_messages([
    ("system", "{system}\n\nLatest News Context:\n{news}"),
    HumanMessage(content="{user_input}")
])

def call_news(inputs):
    news = news_tool.invoke({"limit": inputs.get("limit", 5)})
    return {**inputs, "news": news}


chain = (
    RunnablePassthrough()                 # {'user_input', 'limit'}
    | RunnableLambda(call_news)           # {'user_input', 'limit', 'news'}
    | RunnableLambda(                     # 프롬프트에 채우기
          lambda d: {
              "system": SYSTEM_INSTRUCTION,
              "news": d["news"],
              "user_input": d["user_input"],
          })
    | prompt
    | ChatUpstage(model="solar-pro2")     # 실제 LLM 호출
)

result = chain.invoke({"user_input": input()})
print(result.content)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "user_input": ""
}
[chain/start] [chain:RunnableSequence > chain:RunnablePassthrough] Entering Chain run with input:
{
  "user_input": ""
}
[chain/end] [chain:RunnableSequence > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "user_input": ""
}
[chain/start] [chain:RunnableSequence > chain:call_news] Entering Chain run with input:
{
  "user_input": ""
}
[tool/start] [chain:RunnableSequence > chain:call_news > tool:NewsTool] Entering Tool run with input:
"{'limit': 5}"
[tool/end] [chain:RunnableSequence > chain:call_news > tool:NewsTool] [2.78s] Exiting Tool run with output:
"📰 **Victims of mis-sold car finance could get less than £950 per deal**
🔗 https://www.bbc.com/news/articles/cgjy29zql25o?at_medium=RSS&at_campaign=rss
📄 Victims of car finance mis-selling will likely get less than £950 per deal under a compensation scheme proposed by the financial regulator, with the first payouts expected ne

### LLM이 직접 Tool을 사용하여 뉴스를 가져오기

In [32]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

llm = ChatUpstage(
    model="solar-pro2",
).bind_tools([news_tool])

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "# Overview\n"
     "You are a helpful news bot.\n"
     "Always use the provided tool to fetch fresh news, choosing an appropriate limit.\n"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

agent_runnable = create_tool_calling_agent(
    llm           = llm,
    tools         = [news_tool],
    prompt        = prompt,
)

agent = AgentExecutor(
    agent  = agent_runnable,
    tools  = [news_tool],
    verbose=True,
)

result = agent.invoke({"input": input()})

pprint(result)

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "뉴스 10개 가져와줘"
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": []
}
[chain/end] [

### 각 뉴스에 대한 Summary 후 Keyword, Link 추출을 해봅시다!

In [33]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import Field, BaseModel, RootModel
from typing import List

set_debug(False)

class ArticleSummary(BaseModel):
    summary: str = Field(..., max_length=200)
    keywords: List[str]
    link: str

class Summaries(RootModel[List[ArticleSummary]]):
    pass

json_parser = PydanticOutputParser(pydantic_object=Summaries)


summarize_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are an expert news summarizer.\n"
     "{format_instructions}\n"
     "- Write **one sentence ≤ 25 words** per article (≈ 180 chars max).\n"
     "- Provide 3-5 keywords.\n"
     "- Provide Link\n"),
    ("user", "{articles}")
]).partial(format_instructions=json_parser.get_format_instructions())


summarizer_llm = ChatUpstage(model="solar-pro2", temperature=0)

summarize_chain = summarize_prompt | summarizer_llm | json_parser
pick_articles    = RunnableLambda(lambda r: {"articles": r["output"]})

print("====== SUMMARIZE CHAIN ======")
pprint(summarize_chain)

full_chain = agent | pick_articles | summarize_chain

print("======== FULL CHAIN ========")
pprint(full_chain)

====== SUMMARIZE CHAIN ======
ChatPromptTemplate(input_variables=['articles'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"ArticleSummary": {"properties": {"summary": {"maxLength": 200, "title": "Summary", "type": "string"}, "keywords": {"items": {"type": "string"}, "title": "Keywords", "type": "array"}, "link": {"title": "Link", "type": "string"}}, "required": ["summary", "keywords", "link"], "title": "ArticleSummary", "type": "object"}}, "items": {"$ref": "#/$defs/ArticleSummary"}}\n```'}, messages=[SystemMe

In [34]:
input_text = "최신 7개 뉴스 요약"
result = full_chain.invoke({"input": input_text})



> Entering new AgentExecutor chain...

Invoking: `NewsTool` with `{'limit': 7}`
responded:   
[The NewsTool function is ESSENTIAL to fetch the latest 7 news items as explicitly requested. No other functions or general knowledge can provide real-time news updates, making this the MINIMUM and DIRECTLY NECESSARY call.]

📰 **Victims of mis-sold car finance could get less than £950 per deal**
🔗 https://www.bbc.com/news/articles/cgjy29zql25o?at_medium=RSS&at_campaign=rss
📄 Victims of car finance mis-selling will likely get less than £950 per deal under a compensation scheme proposed by the financial regulator, with the first payouts expected next year.The Supreme Court ruled on Fridaythat hidden commissions from lenders to dealers on car loans were not unlawful, meaning millions of motorists will not be able to claim.However, the judgement left open the possibility of compensation claims for particularly large commissions which the Supreme Court said were unfair.Following the ruling, the F

In [35]:
for res in result.root:
    print(f"Summary: {res.summary}")
    print(f"Keywords: {res.keywords}")
    print(f"Links: {res.link}\n")

Summary: UK car finance victims may get under £950 per deal under a proposed FCA compensation scheme for hidden commissions, costing the industry £9bn-£18bn.
Keywords: ['Car Finance', 'Compensation Scheme', 'FCA', 'Hidden Commissions', 'UK']
Links: https://example.com/car-finance-compensation

Summary: Videos of emaciated Israeli hostages in Gaza spark global outrage; Red Cross seeks access, Israel accuses Hamas of starvation tactics.
Keywords: ['Israeli Hostages', 'Hamas', 'Red Cross', 'Gaza', 'Starvation']
Links: https://example.com/israeli-hostages-gaza

Summary: Proposed Chinese embassy in London faces backlash over security risks, espionage concerns, and proximity to critical infrastructure.
Keywords: ['UK-China Embassy', 'Security Risks', 'Espionage', 'Royal Mint', 'London']
Links: https://example.com/uk-china-embassy-controversy

Summary: A 40-year-old man dies after falling at Oasis's Wembley concert; band and police respond to the tragic incident.
Keywords: ['Oasis Concert', '

#### 문제가 있어요...

Tool 사용을 하면 Link가 그대로 언어모델에게 전달되는데, 한 번 거쳐서 전달되다 보니 Link가 "example.com"으로 떠요.

물론~ 처음부터 tool 사용 가능한 언어모델을 하나만 두고 Output도 구조화된 응답으로 전달하게 하는 chain을 구성하면 됩니다!

아니면 Tool을 직접 사용하지 않게 하고 오로지 패러미터 바인딩만 시키고, Link만 따로 빼낸 후 저장해도 되고요.

간단한 예제여서 이런 상황을 해결하기는 매우 쉽지만, 조금 더 복잡한 경우를 다루어야 할 때에는 **LangGraph**를 사용하는 것이 좋습니다!
